<a href="https://colab.research.google.com/github/KCREEK/Assembly/blob/main/blockchain_simulation_cu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile blockchain_simulation.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <chrono>

#define MAX_BLOCKS 1024 // Max blocks for this example
#define THREADS_PER_BLOCK 256 // Max threads per block

// Structure to define a block in the blockchain
struct Block {
    int blockID;
    long timestamp;
    int transactionCount;
    int previousHash;
    int currentHash;
};

// Kernel to calculate block hashes
__global__ void calculateHash(Block *blocks, int *hashes, int numBlocks) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    if (index < numBlocks) {
        hashes[index] = blocks[index].previousHash ^ blocks[index].transactionCount + blocks[index].timestamp;
        blocks[index].currentHash = hashes[index]; // Update currentHash in the block
    }
}

// Kernel to count the number of blocks in the blockchain
__global__ void countBlocks(int *count, int numBlocks) {
    __shared__ int countCache[THREADS_PER_BLOCK];
    int index = blockIdx.x * blockDim.x + threadIdx.x;

    countCache[threadIdx.x] = (index < numBlocks) ? 1 : 0;
    __syncthreads();

    for (int offset = blockDim.x / 2; offset > 0; offset /= 2) {
        if (threadIdx.x < offset) {
            countCache[threadIdx.x] += countCache[threadIdx.x + offset];
        }
        __syncthreads();
    }

    if (threadIdx.x == 0) {
        atomicAdd(count, countCache[0]);
    }
}

// Function to initialize a dataset of blocks
void initializeBlocks(Block *blocks, int numBlocks) {
    for (int i = 0; i < numBlocks; i++) {
        blocks[i].blockID = i;
        blocks[i].timestamp = 1620000000 + i * 10;
        blocks[i].transactionCount = (i % 10) + 1;
        blocks[i].previousHash = i * 31;
        blocks[i].currentHash = 0;
    }
}

// CPU-based hash calculation (for comparison)
void cpuCalculateHash(Block *blocks, int *hashes, int numBlocks) {
    for (int i = 0; i < numBlocks; i++) {
        hashes[i] = blocks[i].previousHash ^ blocks[i].transactionCount + blocks[i].timestamp;
    }
}

int main() {
    int numBlocks = MAX_BLOCKS;
    int *d_count;
    int host_count = 0;
    Block *blocks = (Block*)malloc(numBlocks * sizeof(Block));
    int *cpu_hashes = (int*)malloc(numBlocks * sizeof(int));

    initializeBlocks(blocks, numBlocks);

    Block *d_blocks;
    int *d_hashes;
    cudaMalloc((void**)&d_blocks, numBlocks * sizeof(Block));
    cudaMalloc((void**)&d_hashes, numBlocks * sizeof(int));
    cudaMalloc((void**)&d_count, sizeof(int));

    // Open file to write output
    FILE *outputFile = fopen("output.txt", "w");
    if (outputFile) {
        fprintf(outputFile, "Blockchain Simulation:\n");
    }

    cudaMemcpy(d_blocks, blocks, numBlocks * sizeof(Block), cudaMemcpyHostToDevice);
    cudaMemcpy(d_count, &host_count, sizeof(int), cudaMemcpyHostToDevice);

    int blocksPerGrid = (numBlocks + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;

    // Measure GPU execution time
    cudaEvent_t startGPU, stopGPU;
    cudaEventCreate(&startGPU);
    cudaEventCreate(&stopGPU);

    cudaEventRecord(startGPU);
    calculateHash<<<blocksPerGrid, THREADS_PER_BLOCK>>>(d_blocks, d_hashes, numBlocks);
    cudaDeviceSynchronize();
    cudaEventRecord(stopGPU);

    cudaEventSynchronize(stopGPU);
    float gpuTime = 0;
    cudaEventElapsedTime(&gpuTime, startGPU, stopGPU);

    // Measure CPU execution time
    auto startCPU = std::chrono::high_resolution_clock::now();
    cpuCalculateHash(blocks, cpu_hashes, numBlocks);
    auto stopCPU = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> cpuTime = stopCPU - startCPU;

    // Copy blocks back to host to get updated currentHash values
    cudaMemcpy(blocks, d_blocks, numBlocks * sizeof(Block), cudaMemcpyDeviceToHost);

    // Write results to file with proper alignment
    if (outputFile) {
        fprintf(outputFile, "GPU Time: %.3f ms\n", gpuTime);
        fprintf(outputFile, "CPU Time: %.3f ms\n", cpuTime.count());
        if (cpuTime.count() > gpuTime) {
            fprintf(outputFile, "CPU runtime is greater than GPU runtime.\n");
        } else {
            fprintf(outputFile, "GPU runtime is greater than CPU runtime.\n");
        }

        // Header with fixed-width format for alignment
        fprintf(outputFile, "\n%-8s %-15s %-17s %-15s %-15s\n",
                "BlockID", "Timestamp", "TransactionCount", "PreviousHash", "CurrentHash");

        // Data rows with the same fixed-width format
        for (int i = 0; i < numBlocks; i++) {
            fprintf(outputFile, "%-8d %-15ld %-17d %-15d %-15d\n",
                    blocks[i].blockID,
                    blocks[i].timestamp,
                    blocks[i].transactionCount,
                    blocks[i].previousHash,
                    blocks[i].currentHash);
        }

        fclose(outputFile);
    }

    // Free allocated memory
    cudaFree(d_blocks);
    cudaFree(d_hashes);
    cudaFree(d_count);
    free(blocks);
    free(cpu_hashes);

    cudaEventDestroy(startGPU);
    cudaEventDestroy(stopGPU);

    return 0;
}


Writing blockchain_simulation.cu


In [2]:
!nvcc blockchain_simulation.cu -o blockchain_simulation


In [ ]:
!./blockchain_simulation


In [ ]:
from google.colab import files
files.download("output.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>